# Calculo de las tablas de contingencia de la precipitación para las estaciones de Hondo, Angela y Pinedo contra los modelos de CHIRPS y MSWEP

In [1]:
%reset -f

import pandas as pd # La usamos para manejar la base de datos (y también graficar) https://pandas.pydata.org/docs/
import xarray as xr
import matplotlib.pyplot as plt # Herramienta principal de visualización https://matplotlib.org/stable/contents.html
import matplotlib.dates as mdates # Dentro de matplotlib, tenemos una herramienta para manejo de fechas 
import seaborn as sbn # Herramienta complementaria de visualización https://seaborn.pydata.org/
import numpy as np # Siempre resulta que la usamos
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score

In [2]:
#cargo los datos de CHIRPS y MSWEP
path = f'{workspace}tesis/datos/procesados/'
mswep_file = 'mswep20192022_validacion.nc'
chirps_file = 'chirps20192022_validacion.nc'
pinedo_file = 'pinedo20192022_validacion.nc'
hondo_file = 'hondo20192022_validacion.nc'
angela_file = 'angela20192022_validacion.nc'


ds_mswep = xr.open_dataset(path+mswep_file, engine='netcdf4')
ds_chirps = xr.open_dataset(path+chirps_file, engine='netcdf4')

# Estos son las series temporales para cada una de las estaciones
ds_pinedo = xr.open_dataset(path+pinedo_file, engine='netcdf4')
ds_hondo = xr.open_dataset(path+hondo_file, engine='netcdf4')
ds_angela = xr.open_dataset(path+angela_file, engine='netcdf4')

In [3]:
ds_chirps = ds_chirps.coarsen(latitude = 2, longitude= 2).mean()
ds_chirps = ds_chirps.where(ds_chirps.precip >1)
ds_mswep = ds_mswep.where(ds_mswep.precipitation >1)

In [4]:
# me creo las estaciones para MSWEP

ds_mswep_pinedo = ds_mswep.sel(lat=slice(-27.1,-27.3)).sel(lon=slice(-61.3,-61.1))
ds_mswep_hondo = ds_mswep.sel(lat=slice(-27.0,-27.2)).sel(lon=slice(-64.5,-64.3))
ds_mswep_angela = ds_mswep.sel(lat=slice(-27.4,-27.5)).sel(lon=slice(-60.9,-60.8))

#ds_mswep_pinedo = ds_mswep_pinedo.where(ds_mswep_pinedo.precipitation > 0.01)
#ds_mswep_hondo = ds_mswep_hondo.where(ds_mswep_hondo.precipitation > 0.01)
#ds_mswep_angela = ds_mswep_angela.where(ds_mswep_angela.precipitation > 0.01)

ds_mswep_pinedo_media = ds_mswep_pinedo.mean(dim='lon', skipna=True).mean(dim='lat', skipna=True)
ds_mswep_hondo_media = ds_mswep_hondo.mean(dim='lon', skipna=True).mean(dim='lat', skipna=True)
ds_mswep_angela_media = ds_mswep_angela.mean(dim='lon', skipna=True).mean(dim='lat', skipna=True)

In [5]:
# me creo las estaciones para CHIRPS

ds_chirps_pinedo = ds_chirps.sel(latitude=slice(-27.2,-27.1)).sel(longitude=slice(-61.3,-61.2))
ds_chirps_hondo = ds_chirps.sel(latitude=slice(-27.3,-27.2)).sel(longitude=slice(-64.5,-64.4))
ds_chirps_angela = ds_chirps.sel(latitude=slice(-27.5,-27.4)).sel(longitude=slice(-60.9,-60.8))

#ds_chirps_pinedo = ds_chirps_pinedo.where(ds_chirps_pinedo.precip > 0.01)
#ds_chirps_hondo = ds_chirps_hondo.where(ds_chirps_hondo.precip > 0.01)
#ds_chirps_angela = ds_chirps_angela.where(ds_chirps_angela.precip > 0.01)


ds_chirps_pinedo_media = ds_chirps_pinedo.mean(dim='longitude', skipna=True).mean(dim='latitude', skipna=True)
ds_chirps_hondo_media = ds_chirps_hondo.mean(dim='longitude', skipna=True).mean(dim='latitude', skipna=True)
ds_chirps_angela_media = ds_chirps_angela.mean(dim='longitude', skipna=True).mean(dim='latitude', skipna=True)

In [6]:
# me creo las estaciones para las estaciones

#ds_pinedo = ds_pinedo.where(ds_pinedo.Precip > 0.1)
#ds_hondo = ds_hondo.where(ds_hondo['Precip avg'] > 0.1)
#ds_angela = ds_angela.where(ds_angela.Precip > 0.1)

In [7]:
#convierto los dataarrays a nparrays

da_pinedo = ds_pinedo.Precip.to_numpy()
da_hondo = ds_hondo['Precip avg'].to_numpy()
da_angela = ds_angela.Precip.to_numpy()

da_pinedo_bin = np.where(da_pinedo == np.nan, np.nan, np.where(da_pinedo>0 ,1,0*da_pinedo))
da_hondo_bin = np.where(da_hondo == np.nan, np.nan, np.where(da_hondo>0 ,1,0*da_hondo))
da_angela_bin = np.where(da_angela == np.nan, np.nan, np.where(da_angela>0 ,1,0*da_angela))
#########################################################################################################
da_chirps_pinedo = ds_chirps_pinedo_media.precip.to_numpy()
da_chirps_hondo = ds_chirps_hondo_media.precip.to_numpy()
da_chirps_angela = ds_chirps_angela_media.precip.to_numpy()

da_chirps_pinedo_bin = np.where(da_chirps_pinedo == np.nan, np.nan, np.where(da_chirps_pinedo>0 ,1,0*da_pinedo))
da_chirps_hondo_bin = np.where(da_chirps_hondo == np.nan, np.nan, np.where(da_chirps_hondo>0 ,1,0*da_hondo))
da_chirps_angela_bin = np.where(da_chirps_angela == np.nan, np.nan, np.where(da_chirps_angela>0 ,1,0*da_angela))

#########################################################################################################
da_mswep_pinedo = ds_mswep_pinedo_media.precipitation.to_numpy()
da_mswep_hondo = ds_mswep_hondo_media.precipitation.to_numpy()
da_mswep_angela = ds_mswep_angela_media.precipitation.to_numpy()

da_mswep_pinedo_bin = np.where(da_mswep_pinedo == np.nan, np.nan, np.where(da_mswep_pinedo>0 ,1,0*da_pinedo))
da_mswep_hondo_bin = np.where(da_mswep_hondo == np.nan, np.nan, np.where(da_mswep_hondo>0 ,1,0*da_hondo))
da_mswep_angela_bin = np.where(da_mswep_angela == np.nan, np.nan, np.where(da_mswep_angela>0 ,1,0*da_angela))

In [8]:
# Cambio el da en un pd.df

df_mswep_pinedo_bin = pd.DataFrame(da_mswep_pinedo_bin)
df_mswep_hondo_bin = pd.DataFrame(da_mswep_hondo_bin)
df_mswep_angela_bin = pd.DataFrame(da_mswep_angela_bin)

df_pinedo_bin = pd.DataFrame(da_pinedo_bin)
df_hondo_bin = pd.DataFrame(da_hondo_bin)
df_angela_bin = pd.DataFrame(da_angela_bin)

df_chirps_pinedo_bin = pd.DataFrame(da_chirps_pinedo_bin)
df_chirps_hondo_bin = pd.DataFrame(da_chirps_hondo_bin)
df_chirps_angela_bin = pd.DataFrame(da_chirps_angela_bin)


In [9]:
df = pd.concat([df_pinedo_bin,df_hondo_bin,df_angela_bin,df_chirps_pinedo_bin,df_chirps_hondo_bin,df_chirps_angela_bin,df_mswep_pinedo_bin,df_mswep_hondo_bin,df_mswep_angela_bin], axis=1)
df.columns = ['df_pinedo_bin','df_hondo_bin','df_angela_bin','df_chirps_pinedo_bin','df_chirps_hondo_bin','df_chirps_angela_bin','df_mswep_pinedo_bin','df_mswep_hondo_bin','df_mswep_angela_bin']
df_nan = pd.DataFrame(df.isna().sum()/len(df)*100).T
df_nan

,df_pinedo_bin,df_hondo_bin,df_angela_bin,df_chirps_pinedo_bin,df_chirps_hondo_bin,df_chirps_angela_bin,df_mswep_pinedo_bin,df_mswep_hondo_bin,df_mswep_angela_bin
0,22.12963,7.962963,32.037037,18.796296,6.666667,26.759259,16.944444,5.462963,25.185185


In [10]:
# Lleno los Nans con el valor mas frecuente

df_mswep_pinedo_bin.fillna(0, inplace=True)
df_mswep_hondo_bin.fillna(0, inplace=True)
df_mswep_angela_bin.fillna(0, inplace=True)

df_pinedo_bin.fillna(0, inplace=True)
df_angela_bin.fillna(0, inplace=True)
df_hondo_bin.fillna(0, inplace=True)

df_chirps_pinedo_bin.fillna(0, inplace=True)
df_chirps_hondo_bin.fillna(0, inplace=True)
df_chirps_angela_bin.fillna(0, inplace=True)

In [11]:
porcentaje_pinedo = df_pinedo_bin.isna().sum()/len(df_pinedo_bin)*100
porcentaje_hondo = df_hondo_bin.isna().sum()/len(df_hondo_bin)*100
porcentaje_angela = df_angela_bin.isna().sum()/len(df_angela_bin)*100


## Matriz de confusion y metricas

||modelo positivo     |modelo negativo|
|----------------|--------------------|---------------|
realidad positivo|verdaderos positivos|falsos negativos    |
realidad negativo|falsos positivos    |verdaderos negativos|

$\text{accuracy score} = \frac{\text{veraderos positivos}}{\text{verdaderos positivos} + \text{falsos positivos}}$  

$\text{recall score} = \frac{\text{veraderos positivos}}{\text{verdaderos positivos} + \text{falsos negativos}}$  

$\text{precision score} = \frac{\text{veraderos positivos} + \text{verdaderos negativos}}{\text{verdaderos positivos} + \text{falsos negativos} + \text{falsos positivos} + \text{verdaderos negativos}}$  


In [12]:
print(50*'-','Pinedo vs CHIRPS',50*'-')
print()
print(pd.DataFrame(confusion_matrix(df_pinedo_bin, df_chirps_pinedo_bin),index=['obs_positivos','obs_negativos'],columns=['modelo_positivo','modelo_negativo']))
print(25*'-', 'estadisticos',25*'-')
print('accuracy: ',accuracy_score(df_pinedo_bin, df_chirps_pinedo_bin))
print('recall: ',recall_score(df_pinedo_bin, df_chirps_pinedo_bin))
print('precision: ', precision_score(df_pinedo_bin, df_chirps_pinedo_bin))
print()

print(50*'-','Hondo vs CHIRPS',50*'-')
print()
print(pd.DataFrame(confusion_matrix(df_hondo_bin, df_chirps_hondo_bin),index=['obs_positivos','obs_negativos'],columns=['modelo_positivo','modelo_negativo']))
print(25*'-', 'estadisticos',25*'-')
print('accuracy: ', accuracy_score(df_hondo_bin, df_chirps_hondo_bin))
print('recall: ', recall_score(df_hondo_bin, df_chirps_hondo_bin))
print('precision: ',precision_score(df_hondo_bin, df_chirps_hondo_bin))
print()

print(50*'-','Angela vs CHIRPS',50*'-')
print()
print(pd.DataFrame(confusion_matrix(df_angela_bin, df_chirps_angela_bin),index=['obs_positivos','obs_negativos'],columns=['modelo_positivo','modelo_negativo']))
print(25*'-', 'estadisticos',25*'-')
print('accuracy: ', accuracy_score(df_angela_bin, df_chirps_angela_bin))
print('recall: ', recall_score(df_angela_bin, df_chirps_angela_bin))
print('precision: ',precision_score(df_angela_bin, df_chirps_angela_bin))
print()

print(50*'-','Pinedo vs MSWEP',50*'-')
print()
print(pd.DataFrame(confusion_matrix(df_pinedo_bin, df_mswep_pinedo_bin),index=['obs_positivos','obs_negativos'],columns=['modelo_positivo','modelo_negativo']))
print(25*'-', 'estadisticos',25*'-')
print('accuracy: ', accuracy_score(df_pinedo_bin, df_mswep_pinedo_bin))
print('recall: ', recall_score(df_pinedo_bin, df_mswep_pinedo_bin))
print('precision: ',precision_score(df_pinedo_bin, df_mswep_pinedo_bin))
print()

print(50*'-','Hondo vs MSWEP',50*'-')
print()
print(pd.DataFrame(confusion_matrix(df_hondo_bin, df_mswep_hondo_bin),index=['obs_positivos','obs_negativos'],columns=['modelo_positivo','modelo_negativo']))
print(25*'-', 'estadisticos',25*'-')
print('accuracy: ', accuracy_score(df_hondo_bin, df_mswep_hondo_bin))
print('recall: ', recall_score(df_hondo_bin, df_mswep_hondo_bin))
print('precision: ',precision_score(df_hondo_bin, df_mswep_hondo_bin))
print()

print(50*'-','Angela vs MSWEP',50*'-')
print()
print(pd.DataFrame(confusion_matrix(df_angela_bin, df_mswep_angela_bin),index=['obs_positivos','obs_negativos'],columns=['modelo_positivo','modelo_negativo']))
print(25*'-', 'estadisticos',25*'-')
print('accuracy: ', accuracy_score(df_angela_bin, df_mswep_angela_bin))
print('recall: ', recall_score(df_angela_bin, df_mswep_angela_bin))
print('precision: ',precision_score(df_angela_bin, df_mswep_angela_bin))

-------------------------------------------------- Pinedo vs CHIRPS --------------------------------------------------

               modelo_positivo  modelo_negativo
obs_positivos              786               76
obs_negativos              139               79
------------------------- estadisticos -------------------------
accuracy:  0.8009259259259259
recall:  0.3623853211009174
precision:  0.5096774193548387

-------------------------------------------------- Hondo vs CHIRPS --------------------------------------------------

               modelo_positivo  modelo_negativo
obs_positivos              848               79
obs_negativos               97               56
------------------------- estadisticos -------------------------
accuracy:  0.837037037037037
recall:  0.3660130718954248
precision:  0.4148148148148148

-------------------------------------------------- Angela vs CHIRPS --------------------------------------------------

               modelo_positivo  modelo_negat

In [13]:
podf_chirps_pinedo = 139/(139+79)
podf_mswep_pinedo = 93/(93+125)


print(podf_chirps_pinedo, podf_mswep_pinedo)


0.6376146788990825 0.42660550458715596
